# Erste Analysen für aggregierte Daten

### Datenimport

Importieren und Zusammenfügen der aggregierten Daten von https://www.ckw.ch/lp/opendata.html 

Aggregierte Daten: Summierter Verbrauch pro Zeitfenster pro Gemeinde

In [1]:
%reset

Nothing done.


In [1]:
# Verwendete Packages:

import numpy as np
import pandas as pd
import os
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.dates as md
%matplotlib inline
import datetime
import ics
from ics import Calendar
import calendar
from urllib.request import urlopen
from csv_ical import Convert
from dateutil import parser
import seaborn as sns
from sklearn import linear_model
import statsmodels.api as sm

Matplotlib is building the font cache; this may take a moment.


#### Import öffentlicher Wetterdaten 
* tre200d0 = °C Lufttemperatur 2 m über Boden; Tagesmittel
* prestad0 = hPa Luftdruck auf Stationshöhe (QFE); Tagesmittel
    

In [14]:
# Import Daten 2023

#df_data = pd.read_pickle("Dataset01.pkl")
df_temp22= pd.read_csv('https://data.geo.admin.ch/ch.meteoschweiz.klima/nbcn-tageswerte/nbcn-daily_LUZ_current.csv', delimiter= ';') # 2021 / 2022
df_temp22=df_temp22[['tre200d0', 'prestad0', 'station/location', 'date']]
df_temp22["date"] = pd.to_datetime(df_temp22["date"], format='%Y%m%d')
df_temp22.rename(columns={'tre200d0':'daytemp','prestad0': "airpress"}, inplace=True)


In [22]:
# Import Daten vor 2023
df_temp21= pd.read_csv('https://data.geo.admin.ch/ch.meteoschweiz.klima/nbcn-tageswerte/nbcn-daily_LUZ_previous.csv', delimiter= ';') # 2021 / 2022
#df_temp21=df_temp21[['tre200d0', 'prestad0', 'station/location', 'date']]
df_temp21["date"] = pd.to_datetime(df_temp21["date"], format='%Y%m%d')
df_temp21.rename(columns={'tre200d0':'daytemp','prestad0': "airpress"}, inplace=True)


In [26]:
df_temp = df_temp21[df_temp21["date"] >= '2021-01-01']
df_temp = df_temp[df_temp["date"] <= '2023-12-31']
df_temp = df_temp[["station/location", "date", "sre000d0", "tre200d0"]]
df_temp.to_csv("weather_daily.csv", index=False)


KeyError: "['tre200d0'] not in index"

In [31]:
import pandas as pd
from io import StringIO

# The provided text
data = """
01.01.2021	Fr	New Year´s Day	53	2
02.01.2021	Sa	Saint Berchtold	53	4
11.02.2021	Th	Carnival Thursday	06	5
14.02.2021	Su	Valentine´s Day	06	5
15.02.2021	Mo	Carnival Monday	07	4
16.02.2021	Tu	Shrove Tuesday	07	5
17.02.2021	We	Ash Wednesday	07	5
28.03.2021	Su	Palm Sunday	12	5
28.03.2021	Su	DST start	12	5
02.04.2021	Fr	Good Friday	13	2
04.04.2021	Su	Easter	13	4
05.04.2021	Mo	Easter Monday	14	4
09.05.2021	Su	Mother's Day	18	5
13.05.2021	Th	Ascension Day	19	2
23.05.2021	Su	Whitsun	20	4
24.05.2021	Mo	Whit Monday	21	4
03.06.2021	Th	Corpus Christi	22	2
06.06.2021	Su	Father´s Day Switzerland	22	5
01.08.2021	Su	Swiss National Holiday	30	2
15.08.2021	Su	Assumption Day	32	2
19.09.2021	Su	Federal Prayday	37	5
02.10.2021	Sa	Saint Leodegar	39	1
31.10.2021	Su	Reformation Day	43	5
31.10.2021	Su	DST end	43	5
31.10.2021	Su	Halloween	43	5
01.11.2021	Mo	All Saints' Day	44	2
02.11.2021	Tu	All Souls' Day	44	5
07.11.2021	Su	Reformation Sunday Switzerland	44	5
28.11.2021	Su	First Advent	47	5
05.12.2021	Su	Second Advent	48	5
06.12.2021	Mo	Saint Nicholas	49	5
08.12.2021	We	Immaculate Conception	49	1
12.12.2021	Su	Third Advent	49	5
19.12.2021	Su	Fourth Advent	50	5
24.12.2021	Fr	Christmas Eve	51	5
25.12.2021	Sa	Christmas	51	2
26.12.2021	Su	St. Stephen´s Day	51	2
31.12.2021	Fr	New Year´s Eve	52	5
01.01.2022	Sa	New Year´s Day	52	2
02.01.2022	Su	Saint Berchtold	52	4
14.02.2022	Mo	Valentine´s Day	07	5
24.02.2022	Th	Carnival Thursday	08	5
28.02.2022	Mo	Carnival Monday	09	4
01.03.2022	Tu	Shrove Tuesday	09	5
02.03.2022	We	Ash Wednesday	09	5
27.03.2022	Su	DST start	12	5
10.04.2022	Su	Palm Sunday	14	5
15.04.2022	Fr	Good Friday	15	2
17.04.2022	Su	Easter	15	4
18.04.2022	Mo	Easter Monday	16	4
08.05.2022	Su	Mother's Day	18	5
26.05.2022	Th	Ascension Day	21	2
05.06.2022	Su	Whitsun	22	4
05.06.2022	Su	Father´s Day Switzerland	22	5
06.06.2022	Mo	Whit Monday	23	4
16.06.2022	Th	Corpus Christi	24	2
01.08.2022	Mo	Swiss National Holiday	31	2
15.08.2022	Mo	Assumption Day	33	2
18.09.2022	Su	Federal Prayday	37	5
02.10.2022	Su	Saint Leodegar	39	1
30.10.2022	Su	DST end	43	5
31.10.2022	Mo	Reformation Day	44	5
31.10.2022	Mo	Halloween	44	5
01.11.2022	Tu	All Saints' Day	44	2
02.11.2022	We	All Souls' Day	44	5
06.11.2022	Su	Reformation Sunday Switzerland	44	5
27.11.2022	Su	First Advent	47	5
04.12.2022	Su	Second Advent	48	5
06.12.2022	Tu	Saint Nicholas	49	5
08.12.2022	Th	Immaculate Conception	49	1
11.12.2022	Su	Third Advent	49	5
18.12.2022	Su	Fourth Advent	50	5
24.12.2022	Sa	Christmas Eve	51	5
25.12.2022	Su	Christmas	51	2
26.12.2022	Mo	St. Stephen´s Day	52	2
31.12.2022	Sa	New Year´s Eve	52	5
01.01.2023	Su	New Year´s Day	52	2
02.01.2023	Mo	Saint Berchtold	01	4
14.02.2023	Tu	Valentine´s Day	07	5
16.02.2023	Th	Carnival Thursday	07	5
20.02.2023	Mo	Carnival Monday	08	4
21.02.2023	Tu	Shrove Tuesday	08	5
22.02.2023	We	Ash Wednesday	08	5
26.03.2023	Su	DST start	12	5
02.04.2023	Su	Palm Sunday	13	5
07.04.2023	Fr	Good Friday	14	2
09.04.2023	Su	Easter	14	4
10.04.2023	Mo	Easter Monday	15	4
14.05.2023	Su	Mother's Day	19	5
18.05.2023	Th	Ascension Day	20	2
28.05.2023	Su	Whitsun	21	4
29.05.2023	Mo	Whit Monday	22	4
04.06.2023	Su	Father´s Day Switzerland	22	5
08.06.2023	Th	Corpus Christi	23	2
01.08.2023	Tu	Swiss National Holiday	31	2
15.08.2023	Tu	Assumption Day	33	2
17.09.2023	Su	Federal Prayday	37	5
02.10.2023	Mo	Saint Leodegar	40	1
29.10.2023	Su	DST end	43	5
31.10.2023	Tu	Reformation Day	44	5
31.10.2023	Tu	Halloween	44	5
01.11.2023	We	All Saints' Day	44	2
02.11.2023	Th	All Souls' Day	44	5
05.11.2023	Su	Reformation Sunday Switzerland	44	5
03.12.2023	Su	First Advent	48	5
06.12.2023	We	Saint Nicholas	49	5
08.12.2023	Fr	Immaculate Conception	49	1
10.12.2023	Su	Second Advent	49	5
17.12.2023	Su	Third Advent	50	5
24.12.2023	Su	Christmas Eve	51	5
24.12.2023	Su	Fourth Advent	51	5
25.12.2023	Mo	Christmas	52	2
26.12.2023	Tu	St. Stephen´s Day	52	2
31.12.2023	Su	New Year´s Eve	52	5


"""

# Use StringIO to simulate a file object
data_io = StringIO(data)

# Read the data into a pandas dataframe
df = pd.read_csv(data_io, sep='\t', header=None, names=["Date", "Day", "Holiday", "Week", "Region"])

# Extract only the date column and save it to a CSV format
date_df = df[["Date"]]


# Output the date dataframe to CSV format
csv_output = date_df.to_csv("holidays.csv", index=False)


enrich data 

In [81]:
smart_meta_df = pd.read_csv("/Users/steffenbeermann/hackday/datahackdays_2024/data_dump/100_15_2023.csv")
smart_meta_df['date'] = pd.to_datetime(smart_meta_df['timestamp']).dt.date
holiday_df = pd.read_csv("holidays.csv")
holiday_df["date"] = pd.to_datetime(holiday_df["date"], format='mixed').dt.date
weather_df = pd.read_csv("weather_daily.csv")
weather_df["date"] = pd.to_datetime(weather_df["date"], format='%Y-%m-%d').dt.date


In [82]:
# combine smart_meta_df with weather_df on column date
df_enriched = pd.merge(smart_meta_df, weather_df, how='left', on='date')
df_enriched

,id,timestamp,value_kwh,year,date,station/location,sunshine_min,daytemp_avg
0,f75148515d279e2c6c36207fa4002faf,2023-03-24T21:45:00.000Z,0.000,2023,2023-03-24,LUZ,0,11.1
1,f75148515d279e2c6c36207fa4002faf,2023-05-29T18:30:00.000Z,0.000,2023,2023-05-29,LUZ,829,18.8
2,f75148515d279e2c6c36207fa4002faf,2023-12-20T01:45:00.000Z,0.000,2023,2023-12-20,LUZ,3,4.0
3,f75148515d279e2c6c36207fa4002faf,2023-11-30T20:15:00.000Z,0.000,2023,2023-11-30,LUZ,0,1.3
4,f75148515d279e2c6c36207fa4002faf,2023-12-30T15:15:00.000Z,0.000,2023,2023-12-30,LUZ,381,4.9
...,...,...,...,...,...,...,...,...
3502770,23f2232b915bf630ab0963b043e5a2be,2023-08-05T19:45:00.000Z,0.073,2023,2023-08-05,LUZ,175,16.3
3502771,09dfebfaf87a26a53a8f87aba08e7ea5,2023-01-31T02:30:00.000Z,0.006,2023,2023-01-31,LUZ,336,3.0
3502772,09dfebfaf87a26a53a8f87aba08e7ea5,2023-12-14T05:30:00.000Z,0.022,2023,2023-12-14,LUZ,0,5.9
3502773,09dfebfaf87a26a53a8f87aba08e7ea5,2023-06-24T11:00:00.000Z,0.019,2023,2023-06-24,LUZ,874,20.8


In [83]:
df_enriched["holiday"] = df_enriched["date"].isin(holiday_df["date"]).astype(int)

In [84]:
df_enriched[df_enriched["holiday"] == 1]
df_enriched.to_csv("100_15_2023_enriched_data.csv", index=False)